# 经典的“策略”模式

UML 类图指出了“策略”模式对类的编排:

![策略模式](source/strategy_pattern.PNG)

[设计模式:可复用面向对象软件的基础]一书是这样概述"策略"模式的:

定义一系列算法,把它们一一封装起来,并且使它们可以相互替换。本模式使得算法可以独立于使用它的客户而变化.

电商领域有个功能明显可以使用“策略”模式,即根据客户的属性或订单中的商品计算折扣.假如一个网店制定了下述折扣规则。


+ 有 1000 或以上积分的顾客,每个订单享 5% 折扣。
+ 同一订单中,单个商品的数量达到 20 个或以上,享 10% 折扣。
+ 订单中的不同商品达到 10 个或以上,享 7% 折扣。

简单起见,我们假定一个订单一次只能享用一个折扣."策略"模式的UML类图,其中涉及下列内容。
+ 上下文

    把一些计算委托给实现不同算法的可互换组件,它提供服务。在这个电商示例中,上下文是 Order,它会根据不同的算法计算促销折扣。

+ 策略

    实现不同算法的组件共同的接口。在这个示例中,名为 Promotion 的抽象类扮演这个角色。

+ 具体策略

    "策略"的具体子类。`fidelityPromo`、`BulkPromo` 和 `LargeOrderPromo`是这里实现的三个具体策略。

In [1]:
from abc import ABC, abstractmethod
from collections import namedtuple

Customer = namedtuple('Customer', 'name fidelity')

class LineItem:
    def __init__(self, product, quantity, price):
        self.product = product
        self.quantity = quantity
        self.price = price
    def total(self):
        return self.price * self.quantity
    
    
class Order:
    """上下文
    """
    def __init__(self, customer, cart, promotion=None):
        self.customer = customer
        self.cart = list(cart)
        self.promotion = promotion
    def total(self):
        if not hasattr(self, '__total'):
            self.__total = sum(item.total() for item in self.cart)
        return self.__total
    def due(self):
        if self.promotion is None:
            discount = 0
        else:
            discount = self.promotion.discount(self)
        return self.total() - discount
    def __repr__(self):
        fmt = '<Order total: {:.2f} due: {:.2f}>'
        return fmt.format(self.total(), self.due())

class Promotion(ABC): 
    """策略:抽象基类"""
    @abstractmethod
    def discount(self, order):
        """返回折扣金额(正值)"""
        pass
    
class FidelityPromo(Promotion): # 第一个具体策略
    """为积分为1000或以上的顾客提供5%折扣""" 
    def discount(self, order):
        return order.total() * .05 if order.customer.fidelity >= 1000 else 0 
    
class BulkItemPromo(Promotion): # 第二个具体策略
    """单个商品为20个或以上时提供10%折扣"""
    def discount(self, order):
        discount = 0
        for item in order.cart:
            if item.quantity >= 20:
                discount += item.total() * .1
        return discount
    
class LargeOrderPromo(Promotion): # 第三个具体策略 
    """订单中的不同商品达到10个或以上时提供7%折扣"""
    def discount(self, order):
        distinct_items = {item.product for item in order.cart}
        if len(distinct_items) >= 10:
            return order.total() * .07
        return 0

In [2]:
joe = Customer('John Doe', 0)
ann = Customer('Ann Smith', 1100)

In [3]:
cart = [LineItem('banana', 4, .5),
        LineItem('apple', 10, 1.5),
        LineItem('watermellon', 5, 5.0)]

In [4]:
Order(joe,cart,FidelityPromo())

<Order total: 42.00 due: 42.00>

In [5]:
Order(ann, cart, FidelityPromo())

<Order total: 42.00 due: 39.90>

In [6]:
banana_cart = [LineItem('banana', 30, .5),
               LineItem('apple', 10, 1.5)]

In [7]:
Order(joe,banana_cart,BulkItemPromo())

<Order total: 30.00 due: 28.50>

In [8]:
long_order = [LineItem(str(item_code), 1, 1.0) for item_code in range(10)]

In [9]:
Order(joe, long_order, LargeOrderPromo())

<Order total: 10.00 due: 9.30>

In [10]:
Order(joe, cart, LargeOrderPromo())

<Order total: 42.00 due: 42.00>


# 使用装饰器优化策略模式

上例完全可用,但是利用 Python 中的装饰器,可以使用更少的代码实现相同的功能.本例除了实现策略模式,还提供了一个选择最佳策略的函数`best_promo`

In [78]:
from abc import ABC, abstractmethod
from collections import namedtuple

Customer = namedtuple('Customer', 'name fidelity')

class LineItem:
    def __init__(self, product, quantity, price):
        self.product = product
        self.quantity = quantity
        self.price = price
    def total(self):
        return self.price * self.quantity
    
    
class Order:
    """上下文
    """
    def __init__(self, customer, cart, promotion=None):
        self.customer = customer
        self.cart = list(cart)
        self.promotion = promotion
    def total(self):
        if not hasattr(self, '__total'):
            self.__total = sum(item.total() for item in self.cart)
        return self.__total
    def due(self):
        if self.promotion is None:
            discount = 0
        else:
            discount = self.promotion(self)
        return self.total() - discount
    def __repr__(self):
        fmt = '<Order total: {:.2f} due: {:.2f}>'
        return fmt.format(self.total(), self.due())

promos = [] 
def promotion(promo_func):  
    promos.append(promo_func) 
    return promo_func

@promotion 
def fidelity(order):
    """为积分为1000或以上的顾客提供5%折扣"""
    return order.total() * .05 if order.customer.fidelity >= 1000 else 0
@promotion
def bulk_item(order):
    """单个商品为20个或以上时提供10%折扣""" 
    discount = 0
    for item in order.cart:
        if item.quantity >= 20:
            discount += item.total() * .1
    return discount
@promotion
def large_order(order):
    """订单中的不同商品达到10个或以上时提供7%折扣""" 
    distinct_items = {item.product for item in order.cart} 
    if len(distinct_items) >= 10:
        return order.total() * .07
    return 0

def best_promo(customer, cart):
    """选择可用的最佳折扣"""
    order = Order(joe, banana_cart )
    max_off,best_promo = max(((promo(order),promo) for promo in promos),
               key=lambda x:x[0])
    return max_off,best_promo.__name__,Order(joe, banana_cart,best_promo)

In [79]:
joe = Customer('John Doe', 0) 
ann = Customer('Ann Smith', 1100)
cart = [LineItem('banana', 4, .5),
        LineItem('apple', 10, 1.5),
        LineItem('watermellon', 5, 5.0)]

In [80]:
Order(ann, cart, fidelity)

<Order total: 42.00 due: 39.90>

In [81]:
banana_cart = [LineItem('banana', 30, .5), 
               LineItem('apple', 10, 1.5)] 

In [82]:
Order(joe, banana_cart, bulk_item) 

<Order total: 30.00 due: 28.50>

In [83]:
long_order = [LineItem(str(item_code), 1, 1.0) for item_code in range(10)]

In [84]:
Order(joe, long_order, large_order)

<Order total: 10.00 due: 9.30>

In [85]:
Order(joe, cart, large_order)

<Order total: 42.00 due: 42.00>

In [86]:
best_promo(joe, banana_cart)

(1.5, 'bulk_item', <Order total: 30.00 due: 28.50>)

In [87]:
best_promo(joe, banana_cart)

(1.5, 'bulk_item', <Order total: 30.00 due: 28.50>)